In [1]:
import os
import numpy as np
from sklearn import mixture
import pandas as pd
import re

In [2]:
#change path
path="..\\UB_keystroke_dataset\\UB_keystroke_dataset"

In [3]:
def ReadListOfTextFilesFromDirectoryWalk(path): 
    files = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if '.txt' in file:
                files.append(os.path.join(r, file))
    return files



In [4]:
files=ReadListOfTextFilesFromDirectoryWalk(path)
len(files)
print(files[0:1])

['..\\UB_keystroke_dataset\\UB_keystroke_dataset\\s0\\baseline\\001000.txt']


In [5]:
def CreateListOfTuplesFromFile(files):
#Reading file and create tuple 
    items = []
    for file in files:
        filename=file.split('\\')[-1].split('.')[0]
        f=open(file, "r")
        for line in f:
            words= line.rstrip('\n').split(sep=" ")
            items.append((words[0],words[1],words[2],filename))
    return items


In [6]:
items=CreateListOfTuplesFromFile(files[0:2])
items

[('LMenu', 'KeyDown', '63578429757751', '001000'),
 ('Tab', 'KeyDown', '63578429757861', '001000'),
 ('LMenu', 'KeyUp', '63578429757970', '001000'),
 ('Tab', 'KeyUp', '63578429757985', '001000'),
 ('LMenu', 'KeyDown', '63578429761511', '001000'),
 ('Tab', 'KeyDown', '63578429761589', '001000'),
 ('LMenu', 'KeyUp', '63578429761698', '001000'),
 ('Tab', 'KeyUp', '63578429761729', '001000'),
 ('LMenu', 'KeyDown', '63578429762229', '001000'),
 ('Tab', 'KeyDown', '63578429762291', '001000'),
 ('LMenu', 'KeyUp', '63578429762416', '001000'),
 ('Tab', 'KeyUp', '63578429762447', '001000'),
 ('LMenu', 'KeyDown', '63578429762837', '001000'),
 ('Tab', 'KeyDown', '63578429762931', '001000'),
 ('Tab', 'KeyUp', '63578429763071', '001000'),
 ('Tab', 'KeyDown', '63578429763789', '001000'),
 ('Tab', 'KeyUp', '63578429763882', '001000'),
 ('LMenu', 'KeyUp', '63578429764085', '001000'),
 ('D0', 'KeyDown', '63578429768578', '001000'),
 ('D0', 'KeyUp', '63578429768656', '001000'),
 ('D0', 'KeyDown', '635784

In [7]:
#Creating dataframes
def CreateDataFrame(items,columns):
    df = pd.DataFrame.from_records(items, columns=['Key', 'EventType','Time','User'])
    return df

In [8]:
df=CreateDataFrame(items,columns=['Key', 'EventType','Time','User'])
print (df.shape)
#
df.head(10)

(13900, 4)


,Key,EventType,Time,User
0,LMenu,KeyDown,63578429757751,001000
1,Tab,KeyDown,63578429757861,001000
2,LMenu,KeyUp,63578429757970,001000
3,Tab,KeyUp,63578429757985,001000
4,LMenu,KeyDown,63578429761511,001000
5,Tab,KeyDown,63578429761589,001000
6,LMenu,KeyUp,63578429761698,001000
7,Tab,KeyUp,63578429761729,001000
8,LMenu,KeyDown,63578429762229,001000
9,Tab,KeyDown,63578429762291,001000


In [9]:
def ParseAlphabetsKeys(df):
    dfAlphabets=df[df["Key"].str.match('^.*[A-Z]$')]
    return dfAlphabets

In [10]:
temp=ParseAlphabetsKeys(df)
temp.groupby("Key").size()/2

Key
A    375.0
B     76.0
C    137.0
D    217.0
E    585.0
F    119.0
G     99.0
H    203.0
I    361.0
J      7.0
K     37.0
L    206.0
M    117.0
N    286.0
O    363.0
P     92.0
Q      4.0
R    262.0
S    322.0
T    453.0
U    149.0
V     57.0
W     92.0
X      6.0
Y    112.0
Z      2.0
dtype: float64

In [11]:
temp

,Key,EventType,Time,User
47,I,KeyDown,63578429792602,001000
48,I,KeyUp,63578429792696,001000
52,A,KeyDown,63578429792961,001000
53,A,KeyUp,63578429793054,001000
54,M,KeyDown,63578429793257,001000
55,M,KeyUp,63578429793382,001000
58,H,KeyDown,63578429793616,001000
59,H,KeyUp,63578429793741,001000
60,O,KeyDown,63578429793756,001000
61,O,KeyUp,63578429793881,001000


In [12]:
def GetTimeDifferenceofKeyDownDown(data):
    prev=0
    rows=[]
    for index,row in data.iterrows():
        if prev==0 and row[1]=="KeyDown":
            prevRow=row
            prev=1
        elif row[1]=="KeyDown":
            rows.append([ prevRow[0], row[0], int(row[2])- int(prevRow[2]),row[3]])
            prevRow=row
    return rows

In [13]:
keyDownDownTimeDifference=GetTimeDifferenceofKeyDownDown(data=temp)
keyDownDownTimeDifference


[['I', 'A', 359, '001000'],
 ['A', 'M', 296, '001000'],
 ['M', 'H', 359, '001000'],
 ['H', 'O', 140, '001000'],
 ['O', 'N', 141, '001000'],
 ['N', 'E', 296, '001000'],
 ['E', 'O', 2980, '001000'],
 ['O', 'R', 62, '001000'],
 ['R', 'E', 78, '001000'],
 ['E', 'D', 515, '001000'],
 ['D', 'T', 281, '001000'],
 ['T', 'O', 187, '001000'],
 ['O', 'B', 250, '001000'],
 ['B', 'E', 187, '001000'],
 ['E', 'W', 343, '001000'],
 ['W', 'I', 140, '001000'],
 ['I', 'T', 63, '001000'],
 ['T', 'H', 1092, '001000'],
 ['H', 'Y', 1357, '001000'],
 ['Y', 'O', 172, '001000'],
 ['O', 'U', 62, '001000'],
 ['U', 'T', 203, '001000'],
 ['T', 'O', 187, '001000'],
 ['O', 'D', 140, '001000'],
 ['D', 'A', 63, '001000'],
 ['A', 'Y', 343, '001000'],
 ['Y', 'A', 1076, '001000'],
 ['A', 'A', 983, '001000'],
 ['A', 'T', 141, '001000'],
 ['T', 'Y', 374, '001000'],
 ['Y', 'O', 187, '001000'],
 ['O', 'U', 78, '001000'],
 ['U', 'R', 141, '001000'],
 ['R', 'C', 405, '001000'],
 ['C', 'O', 109, '001000'],
 ['O', 'M', 172, '0010

In [16]:
def GetDictionaryOfFeatureVectors(keyDownDownTimeDifference):
    """
    Take List of arrays and return dictionary of 26*26 vector for each user
    """
    dictFV=dict()
    for xi in keyDownDownTimeDifference:
        if xi[3] not in dictFV:
            dictFV[xi[3]]=np.zeros(26*26,dtype=object)
            
        index=(ord(xi[0])-65)*26+np.absolute(ord(xi[1])-65)
        if dictFV[xi[3]][index]==0:
            dictFV[xi[3]][index]=[]
        dictFV[xi[3]][index].append(xi[2])
    return dictFV


In [71]:
FeatureVectors=GetDictionaryOfFeatureVectors(keyDownDownTimeDifference) 
usersFV=pd.DataFrame.from_dict(FeatureVectors)
GMMData=[]
for user in usersFV.columns:
    maxLength=0
    for userF in usersFV[user]:
        if userF==0:
            userF=[]
        featureLenght=len(userF)
        if featureLenght > maxLength:
            maxLength=featureLenght
    userDF=pd.DataFrame(index=range(0,676))
    userData=np.zeros((maxLength,676))
    row=0
    for userF in usersFV[user]:
        column=0
        if userF==0:
            userF=[]
        availableValuesCount=len(userF)
        sum=0
        for value in userF:
            userData[column][row]=value
            column=column+1
            sum=sum+value
        if availableValuesCount==0:
            mean=0
        else:
            mean=sum/availableValuesCount
        gussianValues=np.random.normal(mean,3,maxLength-availableValuesCount)
        for value in gussianValues:
            
            userData[column][row]=value
            column=column+1
        
        row=row+1
    print(userData.shape)
    if len(GMMData)==0:
        GMMData=userData
    else:
        GMMData=np.append(GMMData,userData,axis=0)

print(GMMData.shape)

(66, 676)
(48, 676)
(114, 676)


In [70]:
x1='Z'
x2='Z'
(ord(x1)-65)*26+np.absolute(ord(x2)-65)


675

[['I', 'A', 359, '001000'],
 ['A', 'M', 296, '001000'],
 ['M', 'H', 359, '001000'],
 ['H', 'O', 140, '001000'],
 ['O', 'N', 141, '001000'],
 ['N', 'E', 296, '001000'],
 ['E', 'O', 2980, '001000'],
 ['O', 'R', 62, '001000'],
 ['R', 'E', 78, '001000'],
 ['E', 'D', 515, '001000'],
 ['D', 'T', 281, '001000'],
 ['T', 'O', 187, '001000'],
 ['O', 'B', 250, '001000'],
 ['B', 'E', 187, '001000'],
 ['E', 'W', 343, '001000'],
 ['W', 'I', 140, '001000'],
 ['I', 'T', 63, '001000'],
 ['T', 'H', 1092, '001000'],
 ['H', 'Y', 1357, '001000'],
 ['Y', 'O', 172, '001000'],
 ['O', 'U', 62, '001000'],
 ['U', 'T', 203, '001000'],
 ['T', 'O', 187, '001000'],
 ['O', 'D', 140, '001000'],
 ['D', 'A', 63, '001000'],
 ['A', 'Y', 343, '001000'],
 ['Y', 'A', 1076, '001000'],
 ['A', 'A', 983, '001000'],
 ['A', 'T', 141, '001000'],
 ['T', 'Y', 374, '001000'],
 ['Y', 'O', 187, '001000'],
 ['O', 'U', 78, '001000'],
 ['U', 'R', 141, '001000'],
 ['R', 'C', 405, '001000'],
 ['C', 'O', 109, '001000'],
 ['O', 'M', 172, '0010

In [116]:
def GetAverageFeatureVectors(FeatureVectors):
    
    averageFV= np.zeros(26*26,dtype=object)
    index=0
    for xi in FeatureVectors:
        tempSum=np.array(xi).sum()
        if type(xi)== list:
            averageFV[index]=tempSum/len(xi)
        index=index+1
    return averageFV

In [117]:
GetAverageFeatureVectors(FeatureVectors)

array([1104.0, 195.16666666666666, 417.77777777777777, 250.44444444444446,
       1474.5, 312.0, 733.0, 460.0, 244.66666666666666, 0,
       163.83333333333334, 179.65217391304347, 168.2, 208.75862068965517,
       0, 309.5, 0, 210.625, 199.91304347826087, 117.17777777777778,
       468.0, 142.83333333333334, 225.0, 0, 399.875, 0, 0,
       410.3333333333333, 0, 241.61111111111111, 2028.0, 0, 0, 168.4, 0,
       0, 0, 0, 0, 138.28571428571428, 0, 0, 0, 0, 468.0, 179.5, 0, 0, 0,
       224.6, 0, 0, 6006.0, 0, 184.1, 0, 0, 104.0, 415.375, 0, 156.0,
       268.4, 0, 0, 112.9047619047619, 0, 0, 187.0, 514.0, 156.0, 156.0,
       0, 0, 0, 0, 0, 0, 0, 316.6666666666667, 311.9130434782609,
       623.8333333333334, 639.75, 249.5, 456.72222222222223, 0, 0, 203.0,
       397.5, 715.6666666666666, 443.2352941176471, 624.0, 0,
       209.28571428571428, 465.85714285714283, 595.2307692307693, 257.5,
       0, 483.0, 0, 0, 0, 0, 0, 0, 632.2666666666667, 305.0925925925926,
       423.71875, 1225.545

In [72]:
testPath='..\\UB_keystroke_dataset\\UB_keystroke_dataset\\s1\\baseline\\001000.txt'
g = mixture.GMM(n_components=2)
g.fit(GMMData)
np.round(g.weights_, 2)

C:\Users\Muhammad Umsan\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class GMM is deprecated; The class GMM is deprecated in 0.18 and will be  removed in 0.20. Use class GaussianMixture instead.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Muhammad Umsan\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function distribute_covar_matrix_to_match_covariance_type is deprecated; The function distribute_covar_matrix_to_match_covariance_typeis deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Muhammad Umsan\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function log_multivariate_normal_density is deprecated; The function log_multivariate_normal_density is deprecated in 0.18 and will be removed in 0.20.
  warnings.warn(msg, category=DeprecationWarning)
C:\Users\Muhammad Umsan\Anaconda3\lib\site-packages\sklearn\utils\deprec

array([0.58, 0.42])

In [38]:
dic= dict()
dic["abc"]="ABC"

In [39]:
dic["abc"]

'ABC'

In [46]:
if "abc" in dic:
    print("exist")

exist
